# Введение в MapReduce модель на Python


In [110]:
from typing import NamedTuple, Iterator # requires python 3.6+


In [111]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [112]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [113]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [114]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [115]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [116]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [117]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [118]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [119]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [120]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [121]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [122]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [123]:
from typing import NamedTuple, Iterator # requires python 3.6+


class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [124]:

import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.0189403144375144),
 (1, 2.0189403144375144),
 (2, 2.0189403144375144),
 (3, 2.0189403144375144),
 (4, 2.0189403144375144)]

## Inverted index

In [125]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('it', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [126]:


d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [127]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [128]:

import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('it', 18), ('what', 10)]),
 (1, [('a', 2), ('banana', 2), ('is', 18)])]

## TeraSort

In [129]:
# import numpy as np

# input_values = np.random.rand(30)
# maps = 3
# reducers = 2
# min_value = 0.0
# max_value = 1.0

# def INPUTFORMAT():
#   global maps

#   def RECORDREADER(split):
#     for value in split:
#         yield (value, None)

#   split_size =  int(np.ceil(len(input_values)/maps))
#   for i in range(0, len(input_values), split_size):
#     yield RECORDREADER(input_values[i:i+split_size])

# def MAP(value:int, _):
#   yield (value, None)

# def PARTITIONER(key):
#   global reducers
#   global max_value
#   global min_value
#   bucket_size = (max_value-min_value)/reducers
#   bucket_id = 0
#   while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
#     bucket_id += 1
#   return bucket_id

# def REDUCE(value:int, _):
#   yield (None,value)

# partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
# partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
# partitioned_output

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [130]:
from typing import NamedTuple, Iterator

class Number(NamedTuple):
    value: int

input_collection = [
    Number(value=10),
    Number(value=25),
    Number(value=15),
    Number(value=40),
    Number(value=30),
]

# Для каждого числа передаёт саму цифру в качестве ключа
def MAP(_, row: NamedTuple):
    yield ("max", row.value)

# Находит максимальное значение среди всех чисел
def REDUCE(key: str, values: Iterator[int]):
    max_value = max(values)
    yield (key, max_value)

# Преобразует коллекцию в формат, подходящий для MapReduce
def RECORDREADER():
    return [(i, num) for i, num in enumerate(input_collection)]

# Выполнение MapReduce
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('max', 40)]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [131]:
from typing import NamedTuple, Iterator

class Number(NamedTuple):
    value: int

input_collection = [
    Number(value=10),
    Number(value=10)
]

# Для каждого числа передаёт саму цифру и 1, чтобы учитывать количество чисел
def MAP(_, row: NamedTuple):
    yield ("mean", (row.value, 1))

# Суммирует все значения и считает количество
def REDUCE(key: str, values: Iterator[tuple]):
    total_sum = 0
    count = 0
    for value, cnt in values:
        total_sum += value
        count += cnt
    if count > 0:
        mean = total_sum / count
        yield (key, mean)
    else:
        yield (key, 0)

# Преобразует коллекцию в формат, подходящий для MapReduce
def RECORDREADER():
    return [(i, num) for i, num in enumerate(input_collection)]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('mean', 10.0)]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [132]:
from typing import NamedTuple, Iterator

class Number(NamedTuple):
    key: str
    value: int

input_collection = [
    Number(key="A", value=10),
    Number(key="B", value=25),
    Number(key="A", value=15),
    Number(key="C", value=40),
    Number(key="B", value=30),
    Number(key="C", value=5),
]

# Для каждого числа передает его ключ и значение
def MAP(_, row: NamedTuple):
    yield (row.key, row.value)

# Группирует значения по ключам
def REDUCE(key: str, values: Iterator[int]):
    group = list(values)
    yield (key, group)

# Преобразует коллекцию в формат, подходящий для MapReduce
def RECORDREADER():
    return [(i, num) for i, num in enumerate(input_collection)]

# Для имитации MapReduce с сортировкой
def MapReduceWithSorting(RECORDREADER, MAP, REDUCE):
    records = RECORDREADER()

    # Применяется MAP
    mapped = []
    for key, value in records:
        mapped.extend(MAP(key, value))

    # Сортируется по ключу
    mapped.sort(key=lambda x: x[0])

    # Применяется REDUCE
    from itertools import groupby
    output = []
    for key, group in groupby(mapped, key=lambda x: x[0]):
        values = [x[1] for x in group]
        output.extend(REDUCE(key, values))

    return output

# Выполнение MapReduce с сортировкой
output = MapReduceWithSorting(RECORDREADER, MAP, REDUCE)
output = list(output)
output


[('A', [10, 15]), ('B', [25, 30]), ('C', [40, 5])]

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [133]:
from typing import NamedTuple, Iterator
import numpy as np

class Number(NamedTuple):
    key: str
    value: int

input_collection = [
    Number(key="A", value=10),
    Number(key="B", value=25),
    Number(key="A", value=15),
    Number(key="C", value=40),
    Number(key="B", value=30),
    Number(key="C", value=5),
]

maps = 3
reducers = 3

def INPUTFORMAT():
    global maps

    # Функция для чтения записей из "раздела" (split)
    def RECORDREADER(split):
        for index, record in enumerate(split):
            yield (index, record)

    # Определяет размер раздела (split_size) для мапперов
    split_size = int(np.ceil(len(input_collection) / maps))
    # Генерирует разделы для мапперов
    for i in range(0, len(input_collection), split_size):
        yield RECORDREADER(input_collection[i : i + split_size])

#  Возвращает пару (ключ, запись)
def MAP(_, number: Number):
    yield (number.key, number)

# Из всех записей с одинаковым ключом остаётся первая встретившаяся
def REDUCE(key: str, numbers: Iterator[Number]):
    yield next(iter(numbers))

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output


6 key-value pairs were sent over a network.


[(0, [Number(key='B', value=25)]),
 (1, [Number(key='A', value=10)]),
 (2, [Number(key='C', value=40)])]

#Операторы реляционной алгебры



### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.

In [134]:
from typing import NamedTuple, Iterator

class Item(NamedTuple):
    id: int
    value: int

input_collection = [
    Item(id=1, value=10),
    Item(id=2, value=25),
    Item(id=3, value=5),
    Item(id=4, value=30),
    Item(id=5, value=15),
]

# Предикат для выборки: условие на значение
# Пример: выбираем элементы, где значение больше 10
def predicate(t: Item) -> bool:
    return t.value > 10

# Если предикат истинен, возвращает пару (t, t)
def MAP(_, row: NamedTuple):
    if predicate(row):
        yield (row, row)

# Возвращает ключ
def REDUCE(key: NamedTuple, values: Iterator[NamedTuple]):
    for value in values:
        yield value

def RECORDREADER():
    return [(i, item) for i, item in enumerate(input_collection)]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[Item(id=2, value=25), Item(id=4, value=30), Item(id=5, value=15)]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [135]:
def RECORDREADER():
    return [
        ('Alice', 30, 'Engineer', 'USA'),
        ('Bob', 25, 'Designer', 'Canada'),
        ('Charlie', 35, 'Artist', 'UK')
    ]

# Множество атрибутов S (в данном случае имя (0) и профессия (2))
S = {0, 2}

# Функция MAP - исключает атрибуты, не принадлежащие S, и создает пару (t', t')
def MAP(*record):
    t_prime = tuple(record[i] for i in S)
    return (t_prime, t_prime)

# Функция REDUCE - сводит все элементы с одинаковым ключом в одну пару (t', t')
# Для каждого ключа возвращает одну пару (t', t')
def REDUCE(key, values):
    unique_values = set(values)
    return (key, list(unique_values))

result = MapReduce(RECORDREADER, MAP, REDUCE)
print(list(result))

['Alice', ['Engineer'], 'Bob', ['Designer'], 'Charlie', ['Artist']]


### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [136]:
from typing import NamedTuple, Iterator

class Item(NamedTuple):
    id: int
    value: int

input_collection = [
    Item(id=1, value=10),
    Item(id=2, value=25),
    Item(id=3, value=5),
    Item(id=4, value=30),
    Item(id=5, value=15),
]

# Для каждого элемента создает пару (t, t)
def MAP(_, row: NamedTuple):
    yield (row, row)

# Независимо от количества значений возвращает пару (t, t)
def REDUCE(key: NamedTuple, values: Iterator[NamedTuple]):
    for value in values:
        yield (value, value)

def RECORDREADER():
    return [(i, item) for i, item in enumerate(input_collection)]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(Item(id=1, value=10), Item(id=1, value=10)),
 (Item(id=2, value=25), Item(id=2, value=25)),
 (Item(id=3, value=5), Item(id=3, value=5)),
 (Item(id=4, value=30), Item(id=4, value=30)),
 (Item(id=5, value=15), Item(id=5, value=15))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [137]:
from typing import NamedTuple, Iterator

class Item(NamedTuple):
    id: int
    value: int

input_collection = [
    Item(id=1, value=10),
    Item(id=2, value=25),
    Item(id=3, value=5),
    Item(id=4, value=30),
    Item(id=5, value=15),
    Item(id=6, value=25),
    Item(id=7, value=10),
]

# Используем значение как ключ
def MAP(_, row: NamedTuple):
    yield (row.value, row)

# Если значение встречается более одного раза,
# то возвращает все значения, которые соответствуют ключу
def REDUCE(key: int, values: Iterator[NamedTuple]):
    value_list = list(values)
    if len(value_list) > 1:
        for value in value_list:
            yield value

def RECORDREADER():
    return [(i, item) for i, item in enumerate(input_collection)]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
for item in output:
    print(item)

Item(id=1, value=10)
Item(id=7, value=10)
Item(id=2, value=25)
Item(id=6, value=25)


### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [138]:
from typing import NamedTuple, Iterator

class Item(NamedTuple):
    id: int
    value: int

# Отношения R и S
R = [
    Item(id=1, value=10),
    Item(id=2, value=25),
    Item(id=3, value=5),
]

S = [
    Item(id=4, value=30),
    Item(id=5, value=15),
    Item(id=1, value=10),
]

def MAP(collection_id, item: Item):
    yield (item, collection_id)

# Возвращает элемент только если он входит в R, но не в S
def REDUCE(item: Item, collections):
    if collections == [0]:  # 0 соответствует R, 1 соответствует S
        yield item

def RECORDREADER():
    return [(0, r) for r in R] + [(1, s) for s in S]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output


[Item(id=2, value=25), Item(id=3, value=5)]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [139]:
from typing import NamedTuple, Iterator

class RItem(NamedTuple):
    a: int
    b: int

class SItem(NamedTuple):
    b: int
    c: int

# Отношения R и S
R = [
    RItem(a=1, b=10),
    RItem(a=2, b=20),
    RItem(a=3, b=30),
]

S = [
    SItem(b=10, c=100),
    SItem(b=20, c=200),
    SItem(b=30, c=300),
    SItem(b=20, c=250),
]

# Для каждого кортежа (a, b) из R создает пару (b, (R, a))
# Для каждого кортежа (b, c) из S создает пару (b, (S, c))
def MAP(_, row: NamedTuple):
    if isinstance(row, RItem):
        yield (row.b, ("R", row.a))
    elif isinstance(row, SItem):
        yield (row.b, ("S", row.c))

def REDUCE(key: int, values: Iterator[tuple]):
    r_values = []
    s_values = []

    # Разделяет значения на (R, a) и (S, c)
    for value in values:
        if value[0] == "R":
            # Добавление a
            r_values.append(value[1])
        elif value[0] == "S":
            # Добавление c
            s_values.append(value[1])

    # Создает все возможные тройки (a, b, c)
    for a in r_values:
        for c in s_values:
            yield (a, key, c)

def RECORDREADER():
    return [(i, item) for i, item in enumerate(R)] + [(i, item) for i, item in enumerate(S)]


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(1, 10, 100), (2, 20, 200), (2, 20, 250), (3, 30, 300)]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [140]:
from typing import NamedTuple, Iterator

class Item(NamedTuple):
    a: int
    b: int
    c: int

# Входные данные
data = [
    Item(a=1, b=10, c=100),
    Item(a=1, b=20, c=200),
    Item(a=2, b=15, c=150),
    Item(a=2, b=25, c=250),
    Item(a=1, b=30, c=300),
]

# Для каждого кортежа (a, b, c) создает пару (a, b)
def MAP(_, row: NamedTuple):
    yield (row.a, row.b)

def REDUCE(key: int, values: Iterator[int]):
    value_list = list(values)
    result = sum(value_list)

    # Возвращает результат (a, x)
    yield (key, result)

def RECORDREADER():
    return [(i, item) for i, item in enumerate(data)]

# Выполнение MapReduce с операцией SUM
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print("Результат SUM:", output)

Результат SUM: [(1, 60), (2, 40)]


### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [141]:
from typing import NamedTuple, Iterator, List, Tuple

class MatrixElement(NamedTuple):
    row: int
    col: int
    value: float

class VectorElement(NamedTuple):
    index: int
    value: float

# Матрица A (разреженное представление)
A = [
    MatrixElement(row=0, col=0, value=2),
    MatrixElement(row=0, col=1, value=3),
    MatrixElement(row=1, col=0, value=4),
    MatrixElement(row=1, col=1, value=5),
]

# Вектор b (разреженное представление)
b = [
    VectorElement(index=0, value=1),
    VectorElement(index=1, value=2),
]

# Глобальный словарь вектора для удобства
vector_dict = {v.index: v.value for v in b}

# Каждый элемент (i, j, A_ij) передается с ключом i
def MAP(_, row: MatrixElement):
    yield (row.row, (row.col, row.value))

# Вычисляет скалярное произведение строки i на вектор b
def REDUCE(i: int, values: Iterator[Tuple[int, float]]):
    result = 0
    # Умножение происходит только если есть значение вектора b_j
    for j, A_ij in values:
        if j in vector_dict:
            result += A_ij * vector_dict[j]
    yield (i, result)

def RECORDREADER():
    return [(el.row, el) for el in A]

# Запуск MapReduce
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
print("Результат умножения матрицы на вектор:", output)

Результат умножения матрицы на вектор: [(0, 8), (1, 14)]


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [142]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

# Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [143]:
import numpy as np

# Размерности матриц
I = 2
J = 3
K = 4 * 10

# Генерация случайных матриц
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

# Генерирует элементы большой матрицы big_mat построчно
def RECORDREADER():
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            yield ((j, k), big_mat[j, k])

# Получает элементы (j, k, njk) и генерирует (i, k, mij * njk).
def MAP(k1, v1):
    (j, k) = k1  # Индексы строки и столбца big_mat
    w = v1  # Значение njk

    # Перебираются все строки small_mat
    # По ключу (i, k) вычисляется произведение элементов малой и большой матриц mij * njk
    for i in range(small_mat.shape[0]):
        mij = small_mat[i, j]
        yield ((i, k), mij * w)

# Суммирует элементы с одинаковыми ключами (i, k).
def REDUCE(key, values):
    yield (key, sum(values))

# Запуск MapReduce
solution = list(MapReduce(RECORDREADER, MAP, REDUCE))
print("Результат умножения матриц (разреженное представление):")
for (i, k), value in solution:
    print(f"P[{i}, {k}] = {value}")

Результат умножения матриц (разреженное представление):
P[0, 0] = 1.9909768648371624
P[1, 0] = 1.4839327217893072
P[0, 1] = 0.9393285133100293
P[1, 1] = 0.7835556802848624
P[0, 2] = 1.068206570666786
P[1, 2] = 0.891622950447476
P[0, 3] = 0.8568591492471703
P[1, 3] = 0.35157764585871815
P[0, 4] = 1.0276854526028885
P[1, 4] = 0.7287282142917347
P[0, 5] = 0.5848353185756127
P[1, 5] = 0.5559578968012043
P[0, 6] = 1.6455589139617917
P[1, 6] = 1.3099489470098709
P[0, 7] = 0.7996834234505383
P[1, 7] = 0.7317979645898649
P[0, 8] = 1.3821297846071805
P[1, 8] = 0.9242149871736705
P[0, 9] = 1.402252015558998
P[1, 9] = 0.9791408020704214
P[0, 10] = 1.6004287091990368
P[1, 10] = 1.1536370358602142
P[0, 11] = 1.7499920013558619
P[1, 11] = 1.2922279899147107
P[0, 12] = 0.8681565042034252
P[1, 12] = 0.8404881051137794
P[0, 13] = 0.9655880548470549
P[1, 13] = 0.35297025689754186
P[0, 14] = 1.548888337384756
P[1, 14] = 1.1087241140744102
P[0, 15] = 1.06477862450919
P[1, 15] = 1.0509722932622778
P[0, 16]

Проверьте своё решение

In [144]:
# Функция преобразования результата в матрицу
def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i, k), _) in reduce_output) + 1
    K = max(k for ((i, k), _) in reduce_output) + 1
    mat = np.zeros((I, K))

    for ((i, k), vw) in reduce_output:
        mat[i, k] = vw

    return mat

# Проверка результата
reference_solution = np.matmul(small_mat, big_mat)
matrix_result = asmatrix(solution)

print("Проверка:", np.allclose(reference_solution, matrix_result))

Проверка: True


# Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [145]:
import numpy as np

# Размерности матриц
I = 2
J = 3
K = 4 * 10

# Генерация случайных матриц
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

# Генерирует элементы для обеих матриц: small_mat и big_mat
def RECORDREADER():
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            yield (('big', j, k), big_mat[j, k])

    for i in range(small_mat.shape[0]):
        for j in range(small_mat.shape[1]):
            yield (('small', i, j), small_mat[i, j])

# Обрабатывает элементы из обеих матриц и генерирует результат умножения.
def MAP(k1, v1):
    source, i_or_j, k = k1

    # Генерация пар - ((строка, столбец), произведение)
    if source == 'big':
        for i in range(small_mat.shape[0]):
            mij = small_mat[i, i_or_j]
            yield ((i, k), mij * v1)

# Суммирует элементы с одинаковыми ключами (i, k)
def REDUCE(key, values):
    yield (key, sum(values))

# Запуск MapReduce
solution = list(MapReduce(RECORDREADER, MAP, REDUCE))
print("Результат умножения матриц (разреженное представление):")
for (i, k), value in solution:
    print(f"P[{i}, {k}] = {value}")

Результат умножения матриц (разреженное представление):
P[0, 0] = 0.2599188600497855
P[1, 0] = 0.4193481821504058
P[0, 1] = 0.5920585299426786
P[1, 1] = 1.2323158738576225
P[0, 2] = 0.15277996541515196
P[1, 2] = 0.40566587164941076
P[0, 3] = 0.233963879753182
P[1, 3] = 0.4092715054753247
P[0, 4] = 0.5621191773877945
P[1, 4] = 1.1152168777652787
P[0, 5] = 0.9325887472849338
P[1, 5] = 1.6345633645326756
P[0, 6] = 0.7298271109797266
P[1, 6] = 1.3236740728604663
P[0, 7] = 0.6517303850480753
P[1, 7] = 0.9115172605621634
P[0, 8] = 0.4482700627192492
P[1, 8] = 1.073793981862095
P[0, 9] = 1.017861246391572
P[1, 9] = 1.5276167136709273
P[0, 10] = 0.706820127467511
P[1, 10] = 1.1070219868939024
P[0, 11] = 0.673965022621339
P[1, 11] = 1.4785484311773738
P[0, 12] = 0.7788084538593332
P[1, 12] = 0.892820729532794
P[0, 13] = 0.8934259972873209
P[1, 13] = 1.397050327431598
P[0, 14] = 0.7783025300852779
P[1, 14] = 1.2448837851350003
P[0, 15] = 0.7719586588901538
P[1, 15] = 1.3865984303553314
P[0, 16] 

In [146]:
# Функция преобразования результата в матрицу
def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i, k), _) in reduce_output) + 1
    K = max(k for ((i, k), _) in reduce_output) + 1
    mat = np.zeros((I, K))

    for ((i, k), vw) in reduce_output:
        mat[i, k] = vw

    return mat

# Проверка результата
reference_solution = np.matmul(small_mat, big_mat)
matrix_result = asmatrix(solution)

# Сравнение результатов
print("Проверка:", np.allclose(reference_solution, matrix_result))


Проверка: True


# Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [147]:
import numpy as np

I = 2
J = 3
K = 4 * 10

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

maps = 2
reducers = 2

def INPUTFORMAT():
    global maps
    def RECORDREADER(i_range):
        for i in i_range:
            for j in range(J):
                for k in range(K):
                    # Генерируются пары (индекс строки малой матрицы, индекс столбца большой матрицы)
                    # и значение произведения соответствующих элементов
                    yield ((i, k), small_mat[i, j] * big_mat[j, k])

    split_size = int(np.ceil(I / maps))
    for i in range(0, I, split_size):
        yield RECORDREADER(range(i, min(i + split_size, I)))

# Принимает пару (индекс, значение) и возвращает её без изменений
def MAP(pair, value):
    yield pair, value

# Возвращает сумму значений для данного ключа
def REDUCE(key, values):
    yield key, sum(values)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=REDUCE)

# Преобразование в матрицу
solution = []
for output_part in partitioned_output:
    for element in output_part[1]:
        solution.append(element)
solution

80 key-value pairs were sent over a network.


[((0, 1), 1.1517510236771775),
 ((0, 2), 1.0240142588834509),
 ((0, 4), 1.0007479328168578),
 ((0, 5), 0.8609916681155934),
 ((0, 8), 0.9636265633285129),
 ((0, 9), 0.30868590648906696),
 ((0, 11), 1.1045678575350184),
 ((0, 12), 1.0830414367351675),
 ((0, 15), 1.008445124558036),
 ((0, 16), 0.4340587283789236),
 ((0, 18), 1.3480248627283218),
 ((0, 19), 0.5272178364068294),
 ((0, 22), 0.5440303354526794),
 ((0, 23), 0.7323056760360698),
 ((0, 25), 0.6273930518798607),
 ((0, 26), 0.9625651855020533),
 ((0, 29), 0.8485193690153491),
 ((0, 32), 1.2540352351109876),
 ((0, 33), 0.5407168884785034),
 ((0, 36), 0.500749570498008),
 ((0, 39), 1.1110497913446171),
 ((1, 0), 0.180099087933439),
 ((1, 1), 0.7253794805906459),
 ((1, 3), 0.6669489142584013),
 ((1, 4), 0.5394643421913019),
 ((1, 7), 0.48672114104467834),
 ((1, 10), 0.17507149935296615),
 ((1, 11), 0.6660315428226),
 ((1, 14), 0.4684403761847726),
 ((1, 17), 0.062375802993738745),
 ((1, 18), 0.7961250304329964),
 ((1, 21), 0.1944978

In [148]:
# Конвертация результата в numpy матрицу
def asmatrix(pairs):
    matrix = np.zeros((I, K))
    for (i, k), value in pairs:
        matrix[i, k] = value
    return matrix
result_matrix = asmatrix(solution)

# Проверка точности
reference_solution = small_mat @ big_mat
print(np.allclose(reference_solution, result_matrix))

True


# Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [149]:
import numpy as np

# Размерности матриц
I = 2
J = 3
K = 4 * 10

# Генерация случайных матриц
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

# Количество мапперов и редьюсеров
maps = 2
reducers = 2

# Генерация входных данных несколькими RECORDREADER-ами
def INPUTFORMAT():
    def RECORDREADER_SMALL_1():
        for i in range(0, I // 3):
            for j in range(J):
                yield (('small', i, j), small_mat[i, j])

    def RECORDREADER_SMALL_2():
        for i in range(I // 3, I):
            for j in range(J):
                yield (('small', i, j), small_mat[i, j])

    def RECORDREADER_BIG_1():
        for j in range(J):
            for k in range(0, K // 3):
                yield (('big', j, k), big_mat[j, k])

    def RECORDREADER_BIG_2():
        for j in range(J):
            for k in range(K // 3, K):
                yield (('big', j, k), big_mat[j, k])

    # Возвращаются генераторы данных
    return iter([
        RECORDREADER_SMALL_1(),
        RECORDREADER_SMALL_2(),
        RECORDREADER_BIG_1(),
        RECORDREADER_BIG_2(),
    ])

# Генерирует пары ((i, k), произведение).
def MAP(k1, v1):
    source, i_or_j, k = k1

    if source == 'big':
        # Загружаются small_mat локально
        for i in range(I):
            mij = small_mat[i, i_or_j]
            yield ((i, k), mij * v1)

# Суммирует элементы с одинаковыми ключами (i, k)
def REDUCE(key, values):
    yield key, sum(values)

# Распределяет ключи между редьюсерами.
def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers

# Запуск MapReduce
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=REDUCE)
solution = []
for output_part in partitioned_output:
    for element in output_part[1]:
        solution.append(element)
solution

80 key-value pairs were sent over a network.


[((0, 1), 0.8401275673221058),
 ((0, 2), 1.2869981804643742),
 ((0, 4), 1.2848153011208598),
 ((0, 5), 0.8152901029771814),
 ((0, 8), 1.1758593181446244),
 ((0, 9), 0.9395101066661968),
 ((0, 11), 1.25722286012607),
 ((0, 12), 2.026947187777678),
 ((0, 15), 0.9080555777624724),
 ((0, 16), 1.164572928130949),
 ((0, 18), 1.1142790174132142),
 ((0, 19), 1.420988037394878),
 ((0, 22), 1.159798001505809),
 ((0, 23), 1.480931911168756),
 ((0, 25), 1.1315677498166519),
 ((0, 26), 0.9802167275673986),
 ((0, 29), 1.9921491798646997),
 ((0, 32), 1.21403860304645),
 ((0, 33), 1.3651961467113847),
 ((0, 36), 2.21521894944262),
 ((0, 39), 0.9520550413148559),
 ((1, 0), 0.9781063263729515),
 ((1, 1), 0.17841585326405454),
 ((1, 3), 0.18615368197919346),
 ((1, 4), 0.5441788886365505),
 ((1, 7), 0.8067060211858489),
 ((1, 10), 0.8151114445713934),
 ((1, 11), 0.3693734126549809),
 ((1, 14), 0.8579522127753039),
 ((1, 17), 1.1327221381339017),
 ((1, 18), 0.6158617183424937),
 ((1, 21), 0.581499584226606

In [150]:
# Конвертация результата в numpy-матрицу
def asmatrix(pairs):
    matrix = np.zeros((I, K))
    for (i, k), value in pairs:
        matrix[i, k] = value
    return matrix

result_matrix = asmatrix(solution)

# Проверка точности
reference_solution = small_mat @ big_mat
print("Проверка:", np.allclose(reference_solution, result_matrix))

Проверка: True
